In [5]:
%pip install spmf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from spmf import Spmf
import pandas as pd
from itertools import chain, combinations

In [7]:
from itertools import chain, combinations

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def associationRule(freqItemSetDict, itemSetList, minConf):
    rules = []

    # Menghitung dukungan setiap itemset dalam freqItemSet
    itemSetSupport = {}
    for itemSet, support in freqItemSetDict:
        itemSetTuple = frozenset(itemSet)  # Menggunakan frozenset sebagai kunci
        itemSetSupport[itemSetTuple] = support

    for itemSet, support in freqItemSetDict:
        subsets = powerset(itemSet)
        itemSetSup = itemSetSupport[frozenset(itemSet)]  # Mengambil dukungan itemSet

        for s in subsets:
            s_tuple = frozenset(s)
            if s_tuple in itemSetSupport and itemSetSupport[s_tuple] > 0:
                confidence = float(itemSetSup / itemSetSupport[s_tuple])
                if confidence >= minConf:
                    support_B = (itemSetSupport[frozenset(itemSet.difference(s))] / len(itemSetList))
                    
                    lift_ratio = confidence / support_B

                    rules.append([
                        set(s),
                        set(itemSet.difference(s)),
                        support / len(itemSetList) * 100,
                        confidence,
                        support_B,
                        lift_ratio
                    ])

    return rules


In [8]:
# file_path = '../notebook/data/example_spfm_test.txt'

# file_path = '../notebook/data/transaction_01_sd_03_october_spmf_test.txt'
# minsup: 0.0014, minconf: 0.6, minsuppcount: 2.1 

# file_path = '../notebook/data/transaction_01_sd_07_october_spmf_test.txt'
# minsup: 0.0008, minconf: 0.6, minsuppcount: 2.7 

file_path = '../notebook/data/transaction_13_sd_15_october_spmf_test.txt'
# minsup: 0.002, minconf: 0.6, minsuppcount: 2.6

itemSetList = []

with open(file_path, 'r') as file:
    for line in file:
        elements = line.strip().split(' ')
        
        if elements[0].startswith('@'):
            continue  # Lewati baris yang mengandung informasi header
        
        itemSetList.append(elements)


spmf = Spmf("Eclat",
            input_filename=file_path,
            output_filename="../notebook/output/frequent-pattern-eclat.txt",
            arguments=[0.002, False])
spmf.run()

patterns_list = []
with open("../notebook/output/frequent-pattern-eclat.txt", "r") as file:
    for line in file:
        if line.startswith("#SUP:"):
            continue
        parts = line.strip().split(" #SUP: ")
        pattern = frozenset(map(str, parts[0].split()))
        sup = int(parts[1])
        patterns_list.append((pattern, sup))

# Create association rules
rules = associationRule(patterns_list, itemSetList, minConf=0.6)

# Buat DataFrame
columns = ["Antecedent", "Consequent", "Support", "Confidence", "Support B", "Lift"]
df = pd.DataFrame(rules, columns=columns)

# Konversi kolom 'Confidence' ke float
df['Confidence'] = df['Confidence'].astype(float)

# Lakukan pengurutan
df_sorted = df.sort_values(by='Confidence', ascending=False)
df_sorted = df_sorted.reset_index(drop=True)

# Tampilkan DataFrame yang diurutkan
df_sorted

>/D:/dev/project/Skripsi/Skripsi_ECLAT_and_FPGrowth/notebook/spmf.jar
=============  ECLAT v0.96r18 - STATS =============
 Transactions count from database : 1343
 Frequent itemsets count : 382
 Total time ~ 20 ms
 Maximum memory usage : 30.58515167236328 mb
Post-processing to show result in terms of string values.
Post-processing completed.



,Antecedent,Consequent,Support,Confidence,Support B,Lift
0,{8998009720635},{8998009720123},0.297841,1.000000,0.009680,103.307692
1,{8992753184200},{8998009040023},0.223380,0.750000,0.027550,27.222973
2,"{089686010824, 089686010190}",{089686010107},0.223380,0.750000,0.023827,31.476562
3,"{089686010107, 089686010190}",{089686010824},0.223380,0.750000,0.021593,34.732759
4,{711844120037},{711844110113},0.297841,0.666667,0.005212,127.904762
5,{711844120037},{8992770033130},0.297841,0.666667,0.013403,49.740741
6,{8992628022156},{8992770033130},0.223380,0.600000,0.013403,44.766667
7,{8992907952358},{8992907952327},0.223380,0.600000,0.014892,40.290000
